In [1]:
from cipher_operations import (
    init_scheme, hash_column_name, encrypted_data,
    cipher_average, cipher_sum, cipher_inner_product
)
import pandas as pd
import numpy as np
import time

HE = init_scheme()

data = pd.read_csv('data.csv')
data.head()

,Bankrupt,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [2]:
enc_data = encrypted_data(HE, data)
enc_data

{'116532cd6f61d0e2bfbaa7b445591516461e466b43ab6f36976714e58bfc0828': (<Pyfhel Ciphertext at 0x7fc30c629cb0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 'a865215a3bcec1654dec8cac5316f5f469e77c144e2455abe7db070e873115e6': (<Pyfhel Ciphertext at 0x7fc30b933830, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 'ad981008f3abfaae3298f9436b451fc76f7b1dbfac0f10c8963eaff710eb64b2': (<Pyfhel Ciphertext at 0x7fc30b933510, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 '77b63d7feb10a60863fbacace7117d1cae05b6559bcb925b9ebead0eca4b096b': (<Pyfhel Ciphertext at 0x7fc30b933ab0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 'd549c6993df52fad571c185381b3e1315f431641abeffc80386757e68cbc94ce': (<Pyfhel Ciphertext at 0x7fc30b933e20, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 '7b7b0e1b8ffe25afe2575f62927e8e4701dba46ee38d7572a9949f83d0bdb966': (<Pyfhel Ciphertext at 0x7fc30b9323e0, scheme=ckks, size=2/2, scale_bits=30, mod_level

In [3]:
operation_map = {
    'ip': (np.dot, cipher_inner_product),
    'sum' : (np.sum, cipher_sum),
    'avg' : (np.mean, cipher_average),
}

for op_name, (np_op, cipher_op) in operation_map.items():
    print(f'{op_name} performance')
    for column_name in data.columns:
        raw_vector = data[column_name]
        enc_vector, size = enc_data[hash_column_name(column_name)]

        print(column_name)
        print('size:', size)


        start_time = time.perf_counter()
        if op_name in ('sum', 'avg'):
            raw_result = np_op(raw_vector)
        else:
            raw_result = np_op(raw_vector, raw_vector)
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        print('unencrypted operation:', elapsed)

        start_time = time.perf_counter()
        if op_name in ('sum', 'avg'):
            cipher_result = cipher_op(enc_vector, size)
        else:
            cipher_result = cipher_op(enc_vector, enc_vector, size)
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        print('encrypted operation:', elapsed)

        off = (cipher_result.decrypt()[0] - raw_result) / raw_result

        if abs(off) > 0.01:
            print(f'wrong impl {op_name}')
            print(column_name)
            print(f'{off=}')

            break
            
        print('operation result difference:', off)

        print()

ip performance
Bankrupt
size: 6819
unencrypted operation: 0.00016175700875464827
encrypted operation: 0.6552514729992254
operation result difference: -4.8182916145833585e-08

 ROA(C) before interest and depreciation before interest
size: 6819
unencrypted operation: 0.0001318400027230382
encrypted operation: 0.5061448710039258
operation result difference: 1.0399675305261117e-07

 ROA(A) before interest and % after tax
size: 6819
unencrypted operation: 0.00010525999823585153
encrypted operation: 0.49808278800628614
operation result difference: -7.780555526240045e-08

 ROA(B) before interest and depreciation after tax
size: 6819
unencrypted operation: 0.00010811499669216573
encrypted operation: 0.47800605300290044
operation result difference: -3.108512028804913e-08

 Operating Gross Margin
size: 6819
unencrypted operation: 6.933099939487875e-05
encrypted operation: 0.48777586899814196
operation result difference: -1.8742367747625014e-07

 Realized Sales Gross Margin
size: 6819
unencrypted